In [4]:
using JuMP

INFO: Precompiling module JuMP.


In [11]:
# Define model
m = Model()

# Road/Snow removal machine combinations
S = ["11","12","21","22","31","32"]

# Non-negativity
@variable(m, x[S] >= 0)

# Objective function: maximize snow removal
@objective(m, Max, x["11"] + 2x["12"] + 1.2x["21"] + 3x["22"] + 2x["31"] + 3x["32"])

# Each machine can only operate for 24 hours in a day
@constraint(m, x["11"] + x["21"] + x["31"] <= 24)
@constraint(m, x["12"] + x["22"] + x["32"] <= 24)

# Can only plow existing snow from each road
@constraint(m, x["11"] + 2x["12"] <= 15)
@constraint(m, 1.2x["21"] + 3x["22"] <= 24)
@constraint(m, 2x["31"] + 3x["32"] <= 18)

# Solve the optimazation problem
solve(m)

# Determine road/snow machine combinations
println("Variable values: ", getvalue(x))

# Determine optimal amount of snow reamoved
println("Objective value: ", getobjectivevalue(m))

INFO: Precompiling module Clp.


Variable values: x: 1 dimensions:
[11] = 0.0
[12] = 7.5
[21] = 0.0
[22] = 8.0
[31] = 9.0
[32] = 0.0

Objective value: 57.0


In [12]:
# Define model
m2 = Model()

# Road/Snow removal machine combinations
S = ["11","12","21","22","31","32"]

# Non-negativity
@variable(m2, x[S] >= 0)
@variable(m2, z >= 0)

# Minimize snow removal time
@objective(m2, Min, z)

# Each machine can only be on the road the min amount of time
@constraint(m2, x["11"] + x["21"] + x["31"] <= z)
@constraint(m2, x["12"] + x["22"] + x["32"] <= z)

# All the snow must be removed from each road
@constraint(m2, x["11"] + 2x["12"] == 15)
@constraint(m2, 1.2x["21"] + 3x["22"] == 24)
@constraint(m2, 2x["31"] + 3x["32"] == 18)

# Solve the optimazation problem
solve(m2)

# Determine road/snow machine combinations
println("Variable values: ", getvalue(x))

# Determine optimal amount of snow removed
println("Objective value: ", getobjectivevalue(m2))



Variable values: x: 1 dimensions:
[11] = 4.333333333333333
[12] = 5.333333333333334
[21] = 0.0
[22] = 8.0
[31] = 9.0
[32] = 0.0

Objective value: 13.333333333333332


In [13]:
# Define model
diet = Model()

# Non-negativity and define food variables
@variable(diet, x >= 0)
@variable(diet, y >= 0)

# Objective function: minimize cost
@objective(diet, Min, 2x + 3y)

# Set up constraints
# Need to obtain minimum amount of nutrients
# Nutrient 1
@constraint(diet, 4x + 6y >= 30)
# Nutrient 2
@constraint(diet, 6x + 2y >= 20)
# Nutrient 3
@constraint(diet, x + 2y >= 12)

# Solve the optimazation problem
status = solve(diet)

# Determine food amount
println("Variable value x: ", getvalue(x))
println("Variable value y: ", getvalue(y))

# Determine optimal cost of food
println("Objective value: ", getobjectivevalue(diet))
# What is the total quantity of food produced
println("Total amount: ", getvalue(x) + getvalue(y))

Variable value x: 1.5999999999999996
Variable value y: 5.2
Objective value: 18.8
Total amount: 6.8


In [14]:
# Import dataframes package and read in the fooddata csv
using DataFrames
fooddata = readtable("fooddata.csv",skipstart=1,nrows=10,eltypes=[UTF8String,Int64,Int64,Int64,Int64,Int64])
println(fooddata)

INFO: Precompiling module DataFrames.


10×6 DataFrames.DataFrame
│ Row │ Food │ Calories │ Fat │ Saturated_Fat │ Carbohydrate │ Protein │
├─────┼──────┼──────────┼─────┼───────────────┼──────────────┼─────────┤
│ 1   │ "1"  │ 448      │ 1   │ 5             │ 36           │ 2       │
│ 2   │ "2"  │ 305      │ 17  │ 4             │ 40           │ 7       │
│ 3   │ "3"  │ 337      │ 4   │ 5             │ 22           │ 4       │
│ 4   │ "4"  │ 455      │ 18  │ 2             │ 83           │ 5       │
│ 5   │ "5"  │ 314      │ 19  │ 1             │ 76           │ 13      │
│ 6   │ "6"  │ 437      │ 19  │ 5             │ 45           │ 1       │
│ 7   │ "7"  │ 438      │ 3   │ 1             │ 93           │ 0       │
│ 8   │ "8"  │ 413      │ 17  │ 3             │ 77           │ 0       │
│ 9   │ "9"  │ 481      │ 13  │ 5             │ 54           │ 0       │
│ 10  │ "10" │ 150      │ 14  │ 0             │ 22           │ 2       │


  likely near In[14]:3


In [15]:
# Define model
food = Model()

# Food options
F = fooddata[:Food]

# Non-negativity and define food variables
@variable(food, x[F] >= 0)

# Objective function: 
@objective(food, Min, sum([fooddata[i,:Calories]*x[fooddata[i,:Food]] for i in 1:size(fooddata)[1]]))

# Set up constraints
# Need to obtain minimum amount of nutrients
# Need at least 50 fats
@constraint(food, sum([fooddata[i,:Fat]*x[fooddata[i,:Food]] for i in 1:size(fooddata)[1]]) >= 50)
# Need at least 300 carbohydrates
@constraint(food, sum([fooddata[i,:Carbohydrate]*x[fooddata[i,:Food]] for i in 1:size(fooddata)[1]]) >= 300)
# Need at least 60 proteins
@constraint(food, sum([fooddata[i,:Protein]*x[fooddata[i,:Food]] for i in 1:size(fooddata)[1]]) >= 60)
# No more than 20 saturated fats
@constraint(food, sum([fooddata[i,:Saturated_Fat]*x[fooddata[i,:Food]] for i in 1:size(fooddata)[1]]) <= 20)

# Solve the optimazation problem
status = solve(food)

# Determine food amount
println("Variable value: ", getvalue(x))

# Determine optimal cost of food
println("Objective value: ", getobjectivevalue(food))


Variable value: x: 1 dimensions:
[ 1] = 0.0
[ 2] = 0.0
[ 3] = 0.0
[ 4] = 0.0
[ 5] = 4.615384615384616
[ 6] = 0.0
[ 7] = 0.0
[ 8] = 0.0
[ 9] = 0.0
[10] = 0.0

Objective value: 1449.2307692307693


In [70]:
# Import dataframes package and read in the fooddata csv
using DataFrames
datingdata = readcsv("compatibility_score_debug_integers.csv")

20×20 Array{Float64,2}:
 81.0  65.0  43.0  75.0  35.0  16.0  …   3.0   5.0  17.0  95.0   3.0  25.0
 90.0   3.0  38.0  25.0  83.0  79.0     74.0  68.0  39.0  92.0  56.0  29.0
 12.0  84.0  76.0  50.0  58.0  31.0     50.0   4.0  83.0   5.0  88.0  61.0
 91.0  93.0  79.0  69.0  54.0  52.0     47.0   7.0  80.0  73.0  66.0  26.0
 63.0  67.0  18.0  89.0  91.0  16.0     90.0  52.0   6.0  26.0  19.0  82.0
  9.0  75.0  48.0  95.0  28.0  60.0  …  60.0   9.0  39.0  42.0  36.0  98.0
 27.0  74.0  44.0  54.0  75.0  26.0     61.0  81.0  52.0  54.0  46.0  73.0
 54.0  39.0  64.0  13.0  75.0  65.0     85.0  81.0  41.0  94.0  98.0  34.0
 95.0  65.0  70.0  14.0  38.0  68.0     80.0  72.0  65.0  41.0  15.0  58.0
 96.0  17.0  75.0  25.0  56.0  74.0     57.0  14.0  62.0  98.0  85.0  10.0
 15.0  70.0  27.0  84.0   7.0  45.0  …  18.0  65.0  29.0  30.0  64.0  90.0
 97.0   3.0  67.0  25.0   5.0   8.0     23.0  51.0  43.0  70.0  37.0  87.0
 95.0  27.0  65.0  81.0  53.0  22.0     88.0  97.0   1.0  66.0  19.0  81.0
 

In [71]:
println(size(datingdata))

(20,20)


In [77]:
eHarmony = readtable("compatibility_score_integers.csv",header=false)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145,x146,x147,x148,x149,x150,x151,x152,x153,x154,x155,x156,x157,x158,x159,x160,x161,x162,x163,x164,x165,x166,x167,x168,x169,x170,x171,x172,x173,x174,x175,x176,x177,x178,x179,x180,x181,x182,x183,x184,x185,x186,x187,x188,x189,x190,x191,x192,x193,x194,x195,x196,x197,x198,x199,x200,x201,x202,x203,x204,x205,x206,x207,x208,x209,x210,x211,x212,x213,x214,x215,x216,x217,x218,x219,x220,x221,x222,x223,x224,x225,x226,x227,x228,x229,x230,x231,x232,x233,x234,x235,x236,x237,x238,x239,x240,x241,x242,x243,x244,x245,x246,x247,x248,x249,x250,x251,x252,x253,x254,x255,x256,x257,x258,x259,x260,x261,x262,x263,x264,x265,x266,x267,x268,x269,x270,x271,x272,x273,x274,x275,x276,x277,x278,x279,x280,x281,x282,x283,x284,x285,x286,x287,x288,x289,x290,x291,x292,x293,x294,x295,x296,x297,x298,x299,x300,x301,x302,x303,x304,x305,x306,x307,x308,x309,x310,x311,x312,x313,x314,x315,x316,x317,x318,x319,x320,x321,x322,x323,x324,x325,x326,x327,x328,x329,x330,x331,x332,x333,x334,x335,x336,x337,x338,x339,x340,x341,x342,x343,x344,x345,x346,x347,x348,x349,x350,x351,x352,x353,x354,x355,x356,x357,x358,x359,x360,x361,x362,x363,x364,x365,x366,x367,x368,x369,x370,x371,x372,x373,x374,x375,x376,x377,x378,x379,x380,x381,x382,x383,x384,x385,x386,x387,x388,x389,x390,x391,x392,x393,x394,x395,x396,x397,x398,x399,x400,x401,x402,x403,x404,x405,x406,x407,x408,x409,x410,x411,x412,x413,x414,x415,x416,x417,x418,x419,x420,x421,x422,x423,x424,x425,x426,x427,x428,x429,x430,x431,x432,x433,x434,x435,x436,x437,x438,x439,x440,x441,x442,x443,x444,x445,x446,x447,x448,x449,x450,x451,x452,x453,x454,x455,x456,x457,x458,x459,x460,x461,x462,x463,x464,x465,x466,x467,x468,x469,x470,x471,x472,x473,x474,x475,x476,x477,x478,x479,x480,x481,x482,x483,x484,x485,x486,x487,x488,x489,x490,x491,x492,x493,x494,x495,x496,x497,x498,x499,x500,x501,x502,x503,x504,x505,x506,x507,x508,x509,x510,x511,x512,x513,x514,x515,x516,x517,x518,x519,x520,x521,x522,x523,x524,x525,x526,x527,x528,x529,x530,x531,x532,x533,x534,x535,x536,x537,x538,x539,x540,x541,x542,x543,x544,x545,x546,x547,x548,x549,x550,x551,x552,x553,x554,x555,x556,x557,x558,x559,x560,x561,x562,x563,x564,x565,x566,x567,x568,x569,x570,x571,x572,x573,x574,x575,x576,x577,x578,x579,x580,x581,x582,x583,x584,x585,x586,x587,x588,x589,x590,x591,x592,x593,x594,x595,x596,x597,x598,x599,x600,x601,x602,x603,x604,x605,x606,x607,x608,x609,x610,x611,x612,x613,x614,x615,x616,x617,x618,x619,x620,x621,x622,x623,x624,x625,x626,x627,x628,x629,x630,x631,x632,x633,x634,x635,x636,x637,x638,x639,x640,x641,x642,x643,x644,x645,x646,x647,x648,x649,x650,x651,x652,x653,x654,x655,x656,x657,x658,x659,x660,x661,x662,x663,x664,x665,x666,x667,x668,x669,x670,x671,x672,x673,x674,x675,x676,x677,x678,x679,x680,x681,x682,x683,x684,x685,x686,x687,x688,x689,x690,x691,x692,x693,x694,x695,x696,x697,x698,x699,x700,x701,x702,x703,x704,x705,x706,x707,x708,x709,x710,x711,x712,x713,x714,x715,x716,x717,x718,x719,x720,x721,x722,x723,x724,x725,x726,x727,x728,x729,x730,x731,x732,x733,x734,x735,x736,x737,x738,x739,x740,x741,x742,x743,x744,x745,x746,x747,x748,x749,x750,x751,x752,x753,x754,x755,x756,x757,x758,x759,x760,x761,x762,x763,x764,x765,x766,x767,x768,x769,x770,x771,x772,x773,x774,x775,x776,x777,x778,x779,x780,x781,x782,x783,x784,x785,x786,x787,x788,x789,x790,x791,x792,x793,x794,x795,x796,x797,x798,x799,x800,x801,x802,x803,x804,x805,x806,x807,x808,x809,x810,x811,x812,x813,x814,x815,x816,x817,x818,x819,x820,x821,x8

In [81]:
println(eHarmony[1,1])

77.0


In [82]:
# Define model
m = Model()

# Compatibility scores
C = eHarmony

# Non-negativity and define compatibility variables
I = i=1:size(C)[1]
J = 1:size(C)[2]
@variable(m, x[I,J] >= 0)

# Objective function: 
@objective(m, Max, sum{x[i,j]C[i,j], i=I, j=J})

# Set up constraints:
for j = J
    @constraint(m, sum{x[i,j], i=I} == 1)
end

for i = I
    @constraint(m, sum{x[i,j], j=J} == 1)
end

# Solve the optimazation problem
status = solve(m)

# Determine matches
#println("Variable value: ", getvalue(x))

# Determine optimal score
println("Objective value: ", getobjectivevalue(m))

Objective value: 99984.0
